In [1]:
%%capture
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

!pip3 install faiss-gpu-cu12
!pip3 install pgvector
!pip3 install kagglehub
!pip3 install scikit-learn

In [4]:
from huggingface_hub import notebook_login
notebook_login()

# Evaluate Similarity Grouping


In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from evaluation.util import calculate_metrics, calc_bleu

import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
# Models
# meta-llama/Llama-3.2-3B-Instruct
# meta-llama/Meta-Llama-3-8B-Instruct

stem = SentenceTransformerEmbeddingModel(ModelMgr())
lsv = LLaMAValidationModel(ModelMgr(), model_path = "meta-llama/Llama-3.2-3B-Instruct")
# lsv = DeepSeekValidationModel(m)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [6]:
ZERO_SHOT_SYSTEM_PROMPT = "You are a validator. Respond with \"no\" and \"yes\" only!"
ZERO_SHOT_PROMPTING_TEMPLATE = 'Does "{}" describes "{}"'

In [7]:
path = kagglehub.dataset_download("uciml/zoo-animal-classification")

100%|██████████| 1.85k/1.85k [00:00<00:00, 4.01MB/s]

Extracting files...


In [8]:
classes = pd.read_csv(f"{path}/class.csv", index_col=0)[["Class_Type"]]
classes

,Class_Type
Class_Number,
1,Mammal
2,Bird
3,Reptile
4,Fish
5,Amphibian
6,Bug
7,Invertebrate


In [9]:
transforms = {"is": "is not", "has": "has no", "does": "does not", "lays": "does not lays", "gives": "gives no"}
transform_cols = {"hair": "has", "feathers": "has", "eggs": "lays", "milk": "gives", "airborne": "is", "aquatic": "is", "predator": "is", "toothed": "is", "backbone": "has", "breathes": "does", "venomous": "is", "fins": "has", "legs": "has", "tail": "has", "domestic": "is", "catsize": "is" }

columns = ["name"] + [col for col in transform_cols]

print(columns)

zoo = pd.read_csv(f"{path}/zoo.csv", index_col=0)
for col in transform_cols:
    t = transform_cols[col]
    zoo[col] = zoo[col].apply(lambda x: f'{t if x else transforms[t]} {col}')
zoo

['name', 'hair', 'feathers', 'eggs', 'milk', 'airborne', 'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize']


,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
animal_name,,,,,,,,,,,,,,,,,
aardvark,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has no tail,is not domestic,is catsize,1
antelope,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is not predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1
bass,has no hair,has no feathers,lays eggs,gives no milk,is not airborne,is aquatic,is predator,is toothed,has backbone,does not breathes,is not venomous,has fins,has no legs,has tail,is not domestic,is not catsize,4
bear,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has no tail,is not domestic,is catsize,1
boar,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wallaby,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is not predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1
wasp,has hair,has no feathers,lays eggs,gives no milk,is airborne,is not aquatic,is not predator,is not toothed,has no backbone,does breathes,is venomous,has no fins,has legs,has no tail,is not domestic,is not catsize,6
wolf,has hair,has no feathers,does not lays eggs,gives milk,is not airborne,is not aquatic,is predator,is toothed,has backbone,does breathes,is not venomous,has no fins,has legs,has tail,is not domestic,is catsize,1


In [10]:
gt = {tuple([key] + [x[col] for col in columns[1:]] + [classes.loc[x["class_type"]].values[0]]) for key, x in zoo.iterrows()}
print(str(gt)[0:500], "...", len(gt))

{('housefly', 'has hair', 'has no feathers', 'lays eggs', 'gives no milk', 'is airborne', 'is not aquatic', 'is not predator', 'is not toothed', 'has no backbone', 'does breathes', 'is not venomous', 'has no fins', 'has legs', 'has no tail', 'is not domestic', 'is not catsize', 'Bug'), ('wallaby', 'has hair', 'has no feathers', 'does not lays eggs', 'gives milk', 'is not airborne', 'is not aquatic', 'is not predator', 'is toothed', 'has backbone', 'does breathes', 'is not venomous', 'has no fins ... 101


In [11]:
data = [x[:-1] for x in gt]
print(data[0])

('housefly', 'has hair', 'has no feathers', 'lays eggs', 'gives no milk', 'is airborne', 'is not aquatic', 'is not predator', 'is not toothed', 'has no backbone', 'does breathes', 'is not venomous', 'has no fins', 'has legs', 'has no tail', 'is not domestic', 'is not catsize')


In [12]:
overall_result = {}

def evaluate(method, threshold, system_prompt=ZERO_SHOT_SYSTEM_PROMPT, prompt_template=ZERO_SHOT_PROMPTING_TEMPLATE):
    pred = []
    runtimes = []
    for animal_type in tqdm(classes.values):
        animal_type = animal_type[0]

        d = Dummy("animals", columns, data)
        s = Select(d, SoftEqual(columns, Constant(animal_type), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        result = s.open().fetch_all()
        toc = time.time()

        pred.extend([tuple([x[col] for col in columns] + [animal_type]) for x in result])
        runtimes.append(toc - tic)

    scores = calculate_metrics(gt, set(pred), np.mean(runtimes))

    print(method, threshold, scores["F1 Score"])

    return scores, pred

In [13]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh)
    overall_result[("threshold", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 7/7 [00:11<00:00,  1.64s/it]


threshold 0.1 0.2506203473945409


100%|██████████| 7/7 [00:10<00:00,  1.48s/it]


threshold 0.2 0.2717086834733894


100%|██████████| 7/7 [00:10<00:00,  1.47s/it]


...

threshold 0.4 0.386046511627907


100%|██████████| 7/7 [00:10<00:00,  1.47s/it]


threshold 0.5 0.32653061224489793


100%|██████████| 7/7 [00:10<00:00,  1.48s/it]

threshold 0.6 0


In [14]:
res, pred = evaluate("zero-few-shot", None)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 7/7 [00:36<00:00,  5.21s/it]

zero-few-shot None 0.6193548387096773


In [15]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh)
    overall_result[("both", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 7/7 [00:44<00:00,  6.41s/it]


both 0.1 0.6193548387096773


100%|██████████| 7/7 [00:40<00:00,  5.72s/it]


both 0.2 0.5921052631578948


100%|██████████| 7/7 [00:37<00:00,  5.30s/it]


...

both 0.4 0.5540540540540542


100%|██████████| 7/7 [00:12<00:00,  1.83s/it]


both 0.5 0.20689655172413793


100%|██████████| 7/7 [00:10<00:00,  1.47s/it]

both 0.6 0


In [16]:
for key in tqdm(overall_result):
    scores_bleu = calc_bleu(gt, overall_result[key]["pred"])
    for score_bleu in scores_bleu:
        overall_result[key][score_bleu] = scores_bleu[score_bleu]

100%|██████████| 13/13 [03:34<00:00, 16.47s/it]


In [20]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results

,method,threshold,Precision,Recall,F1 Score,tp,fn,fp,runtime,pred,bleu1,bleu2,bleu3,bleu4
0,threshold,0.1,0.143262,1.000000,0.250620,101,0,604,1.637721,"{(puma, has hair, has no feathers, does not la...",1.000000,1.000000,1.000000,1.000000
1,threshold,0.2,0.158238,0.960396,0.271709,97,4,516,1.474312,"{(puma, has hair, has no feathers, does not la...",0.999363,0.999358,0.999353,0.999347
2,threshold,0.3,0.168478,0.920792,0.284839,93,8,459,1.469729,"{(haddock, has no hair, has no feathers, lays ...",0.998727,0.998717,0.998706,0.998695
3,threshold,0.4,0.252280,0.821782,0.386047,83,18,246,1.447116,"{(crab, has no hair, has no feathers, lays egg...",0.997065,0.997041,0.997015,0.996990
4,threshold,0.5,0.521739,0.237624,0.326531,24,77,22,1.464199,"{(moth, has hair, has no feathers, lays eggs, ...",0.981684,0.975662,0.970362,0.965376
5,threshold,0.6,0.000000,0.000000,0.000000,0,101,0,1.479953,{},-1.000000,-1.000000,-1.000000,-1.000000
6,zero-few-shot,NaN,0.888889,0.475248,0.619355,48,53,6,5.207931,"{(moth, has hair, has no feathers, lays eggs, ...",0.987973,0.981381,0.975722,0.970887
7,both,0.1,0.888889,0.475248,0.619355,48,53,6,6.405074,"{(moth, has hair, has no feathers, lays eggs, ...",0.987973,0.981381,0.975722,0.970887
8,both,0.2,0.882353,0.445545,0.592105,45,56,6,5.720531,"{(moth, has hair, has no feathers, lays eggs, ...",0.987343,0.980742,0.975078,0.970237
9,both,0.3,0.877551,0.425743,0.573333,43,58,6,5.300844,"{(moth, has hair, has no feathers, lays eggs, ...",0.987018,0.980423,0.974756,0.969912


In [21]:
df_results.to_pickle("Animals_mpnetBaseV2_LLama3B.pkl")
df_results.drop(columns=["pred"]).to_csv("Animals_mpnetBaseV2_LLama3B.csv")